# preprocessing (전처리)
- Data cleaning
- Data Encoding         : 텍스트 데이터 → 숫자로 변환 (범주형 데이터)
- Data Scaling          : 숫자값 정규화
- Outlier               : 이상치
- Feature Engineering   : 속성 생성/수정/가공

# Data Encoding

### Label Encoder
- 범주형 데이터에 대해 적절히 숫자로 변환하는 것

##### one-hot Encoder
- 주어진 데이터를 희소배열로 변환 (One-vs-Rest 배열)
- 희소배열이란 대부분이 0이고 특정 인덱스만 값을 가지고 있는 배열

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import LabelEncoder

items = ['TV', '냉장고', '세탁기', '컴퓨터', '전기난로', '컴퓨터', 'TV', '믹서기', '컴퓨터']

encoder = LabelEncoder()

encoder.fit(items)
encoded_items = encoder.transform(items)
len(encoded_items)

### One hot Encoder
- 주어진 데이터를 희소배열로 변환 (One-vs-Rest 배열)
- 희소배열이란 대부분이 0이고 특정 인덱스만 값을 가지고 있는 배열

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 2차원 형태로 변환
items = np.array(items).reshape(-1, 1)

# One-hot encoding
encoder = OneHotEncoder()

encoder.fit(items)      # 중복값을 제거, 오름차순 정렬 -> 그 인데스에만 1을 준 희소행렬
oh_items = encoder.transform(items)
print(oh_items.toarray())

In [ ]:
encoder.categories_

- DataFrame에서 One-hot encoding 하기

In [ ]:
df = pd.DataFrame({
    'items': ['TV', '냉장고', '세탁기', '컴퓨터', '전기난로', '컴퓨터', 'TV', '믹서기', '컴퓨터']
    })
df

In [32]:
df_dummies = pd.get_dummies(df, dtype=int)

In [ ]:
# DataFrame -> ndarray 변환
# df_dummies.to_numpy()
np.array(df_dummies)

### Data Scaling (Feature Scaling)

##### 표준정규화 (StandardScaler)
- 평균이 0, 표준편차가 1인 값으로
- 이상치에 덜 민감하고, 선형회귀 및 로지스틱 회귀 등의 알고리즘에 적합
- 데이터가 정규분포인 경우 더욱 적합함

##### 최소최대정규화 (MinMaxScaler)
- 0~1 사이의 값으로 변환
- SVM 및 KNN과 같은 거리 기반 모델에 적합
- 이상치에 민감하게 반응, 이상치가 있는 경우 데이터 왜곡 가능성

### Data Scaling (Feature Scaling)
- scaling 작업은 train 데이터, test 데이터에 동일하게 적용해야 함
- fit(): train 데이터
- transform(): train 데이터, test 데이터

In [ ]:
from sklearn.datasets import load_iris

iris_ds = load_iris()
print(iris_ds.feature_names)
iris_ds.data

In [ ]:
from sklearn.preprocessing import StandardScaler
standard_sc = StandardScaler()
standard_sc.fit(iris_ds.data)
standard_sc.transform(iris_ds.data)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax_sc = MinMaxScaler()
# minmax_sc.fit_transform([[20], [30], [40]])
minmax_sc.fit(iris_ds.data)
minmax_sc.transform(iris_ds.data)

### 타이타닉 생존율 예측에 필요한 전처리 해보기

In [ ]:
df = pd.read_csv('./data/titanic.csv')
df.info()

In [62]:
# 전처리 -> 함수
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

def fillna(df):
    """
    결측치 처리 함수
    - Age: 평균치로 대체
    - Cabin: 'N' 기본값으로 대체
    - Emberked: 'N' 기본값으로 대체
    """
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Embarked'] = df['Embarked'].fillna('N')
    return df

def drop_feature(df):
    """
    모델 훈련과 관련 없는 속성 제거:
    - passengerId, Name, Ticket
    """
    return df.drop(['PassengerId', 'Name', 'Ticket'], axis=1) 

def encode_feather(df):
    """
    범주형 데이터를 숫자로 인코딩
    - Sex, Cabin, Embarked
    """
    df['Cabin'] = df['Cabin'].str[:1]
    categories = ['Sex', 'Cabin', 'Embarked']
    for cate_item in categories:
        label_encoding = LabelEncoder()
        df[cate_item] = label_encoding.fit_transform(df[cate_item])
    return df

def scailing_feature(train_data, test_data):
    """
    특성 스케일링
    """
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_data)
    test_scaled = scaler.transform(test_data)
    return train_scaled, test_scaled

def preprocess_data(df):
    df = drop_feature(df)
    df = fillna(df)
    df = encode_feather(df)

    return df

In [58]:
# 전처리 함수 호출
df = preprocess_data(df)

In [61]:
# 훈련-테스트 데이터 분리
from sklearn.model_selection import train_test_split

# 입력-라벨 데이터 분리
titanic_input = df.drop(['Survived'], axis=1)
titanic_label = df['Survived']

x_train, x_test, y_train, y_test = train_test_split(titanic_input, titanic_label, test_size=.2, random_state=0)

In [64]:
# 특성 스케일링
x_scaled_train, x_scaled_test = scailing_feature(x_train, x_test)

In [ ]:
# logisticRegression 훈련
from sklearn.linear_model import LogisticRegression

lr_classifier = LogisticRegression()
lr_classifier.fit(x_scaled_train, y_train)

In [ ]:
# 평가
lr_classifier.score(x_scaled_train, y_train), lr_classifier.score(x_scaled_test, y_test)